# Automate IOS-XR Zero Touch Provisioning

## Overview

On-boarding devices with the initial configuration (day-0 configuration) is a task which needs to be automated in many use-cases, especially if a large number of devices are involved. By limiting the human interaction in this process as much as possible service providers can reduce “truck-offload to service-activation” time, avoid to send skilled personal to the field to configure devices manually and achieve better deployment predictability.

For the management of day-1...day-n configurations we usually use the Cisco Network Services Orchestrator (NSO), so in the majority of use-cases the main goal of the day-0 configuration would be to bootstrap the devices and provide them with the minimal configuration needed to be able to onboard them in Cisco NSO - this means basically setting an IP address, user, password, generate some keys. Of course, we are able to manage more complex configurations if the use-case demands it, but it is recommended to manage as much as possible from the configuration using Cisco NSO, if available. 

Note that if the NED supports sending CLI commands to the device (using the life-status actions) it is also possible and straightforward to implement the MOPs (using workflows or scripts) which will use NSO to perform a software upgrade on the device to the desired version.

There are 3 main protocols supported by Cisco products for automating the day-0 configuration provisioning, depending on the type of device:

•	Cisco ZTP (Zero Touch Provisioning) - available on IOS-XR devices

•	Cisco PnP (Plug and Play) - available on branch devices (IOS/IOS-XE devices)

•	POAP (PowerOn Auto Provisioning) - available on the Nexus switches

This guide demonstrates the use of Zero Touch Provisioning Automation in the context of Cisco IOS-XR. There is also a brief description of the other protocols at the end of the lab guide as well as an example of using the PNP protocol with the NSO PNP server for an IOS-XE router.


## Cisco ZTP

<img src="img/ztp_1.png" width="800"/>

On IOS-XR, ZTP process starts at boot time and scans the configuration for the presence of a username. 
If there is no username configured, ZTP starts a DHCP client on the management interface for IPv4 and IPv6 simultaneously, and wait for a response. Together with the IP address the DHCP server will provide an URL to download a config file or a script using option 67 (or 59 for IPv6). We are using http URLs but other protocols like TFTP are also supported. 

The ZTP process will download the file and based on the first line of the text file received will: 
•	apply the configuration commands it contains (if it starts with ”!! IOS XR”)
•	assume this is a shell script and start the execution  (if it starts with ”#!/bin/bash” or ”#!/bin/sh”) 
•	start the execution if it is python script

If the text file cannot be interpreted as a script or a configuration file, or is bigger than 100MB, ZTP will erase the file and terminate its execution.

Using the different DHCP options/functions available on the server it is possible to dynamically create the file URL based on specific device information like the serial number. In this way we will be able to identify the device which made the request on the server side and provide a customized day0 configuration/script.

Using a script is the most flexible solution since inside the script it is possible to download other config files and apply them (using curl or wget for example / python libraries), implement logic, install packages and so on. In order to notify the provisioning server that the device has finished applying its configuration usually we make a HTTP request as the last command in the script which will signalize to our server that the device finished applying the config.

> The description in this section relates to Classic ZTP process
> supported on IOS-XR. In addition, IOS-XR Release 7 introduces a
> enhanced process called Secure ZTP (defined by Yang-Models to
> implement interactions between the router and a RESTCONF based ZTP
> Server) as well as support for YANG XML payload in the ZTP files,
> these enhancements are out of the scope of the current lab.

## LAB Details

This section includes the existing components of the dCloud session used for the lab. Th IP addresses and credentials of all the components used
may be found in the table below and in the Servers tab of the dCloud session:
 
 
| Name | Description | IP Address | Username | Password |
|--|--|--|--|--|
| Workstation | Windows 7 Server with Remote Desktop | 198.18.133.252 | Administrator | C1sco12345 |
| NSO | NSO	Cisco NSO Server | 198.18.134.50 | cisco | cisco |
| DHCP Server | ISC DHCP Server | 198.18.1.30 | cisco | cisco |
| XR9Kv | Cisco IOS-XR | 198.18.1.32 | cisco | cisco |
| CSR1Kv | Cisco IOS-XE | 198.18.1.31 | cisco | cisco |

<img src="img/ztp_topo_cl.png" width="800"/>

It will enable to understand and use ZTP on IOS-XR devices, to develop a prototype application in Python supporting automatic onboarding of XR devices in Cisco NSO as well as updating a Webex Team space about the onboarding status.

Specific focus will be provided on:

•	IOS-XR ZTP process and scripting capabilities

•	ISC DHCP server configuration

•	Cisco NSO actions capabilities

•	Cisco NSO notification capabilities

•	HTTP server Python application development

•	NETCONF/REST Python application development 

•	Webex Team REST API


<img src="img/ztp_cf.png" width="1000"/>

## DHCP Server configuration

[DHCP playbook](./ztp_dhcp_server.ipynb)

## NSO Service

[NSO Service playbook](./ztp_nso_service.ipynb)


## HTTP Server

[HTTP Server playbook](./ztp_http_server.ipynb)

## Notification Server

[NOTIFICATION Server playbook](./ztp_notification_server.ipynb)


### Start the ZTP process on the XR router

After finishing the DHCP, NSO, HTTP and NOTIFICATION playbooks we are now ready to start the ZTP process.

#### TASK

Open a new terminal window (File -> New ->Terminal) or reuse an existing one

Telnet to the serial console port of the IOS-XR device:

[cisco@nso xr-ztp-lab]$ **telnet virl.demo.dcloud.cisco.com 17003**

Authenticate with username cisco, password cisco

Manually initiate the ztp porocess on the Mgmt Interface. The ztp process auto startts on device boot, but to avoid waiting for the device to reboot we can also manually start the ztp porocess using the ztp command

RP/0/RP0/CPU0:xr#**ztp initiate interface MgmtEth 0/RP0/CPU0/0 debug**

Wait for the ztp process to terminate and see the webex notification arriving.

To exit the telnet session at the end: Ctrl + 5 and then quit<Enter>


### Check that the device was added and synced in NSO

#### TASK

Open a new terminal window (File -> New ->Terminal)

```
[cisco@nso xr-ztp-lab]$ ncs_cli -u admin -C

admin connected from 127.0.0.1 using console on nso
admin@ncs# show running-config devices device 36e6df2c 
devices device 36e6df2c
 address   198.18.1.32
 ...
```

## (Extra) Other zero touch provisioning cisco protocols

### Cisco PNP (IOS/IOS-XE)

On IOS-XE, the PNP client existing on the device will contact the pre-configured PnP server and send at regular configurable  intervals the so-called "work-requests". The server will check if there are some tasks still to be performed on the client (like sending a config file) and responds accordingly. If the device is not in the server mapping database, or if there are no additional tasks to be performed it sends a "back-off" response which basically instructs the device to wait the server defined back-off timeout before sending a new work request. 

A similar call flow as the one in the workshop when using PNP

<img src="img/pnp_cf.png" width="1000"/>


#### DHCP

An important step in the process is the discovery of the PnP server address. This could be done manually  - by configuring the pnp client on the device  - or automated, using different methods - when the device boots, the absence of any startup configuration on the NVRAM triggers the PnP discovery agent to acquire the IP address of the PnP server. In order to acquire the IP address of the PnP server, the PnP agent goes through one of the following discovery mechanisms:

•	DHCP discovery

•	DNS discovery – pnp client builds URLs from the domain name provided by the DHCP server (pnpserver.domainname, ntpserver.domainname)

•	Cloud discovery – try to connect to devicehelper.cisco.com

For example for DHCP discovery the configuration of the option 43 on the DHCP server to provide the address of the PNP server to the corresponding DHCP client(s) will look like:

```
host csrv1k_mgmt {
    hardware ethernet 5e:00:00:01:00:00;
    fixed-address 198.18.1.31;
    option vendor-encapsulated-options "5A1N;K4;B2;I198.18.134.50;J9191";
}
````

Where 

5A1N 	:	PnP DHCP ID, version 1 and debug off (5A1D for debug)

K4	:	HTTP protocol (without ssl, K5 with SSL )

B2	:	Address Type ipv4 (B1 for URL)

I…	:	Address

J…	: 	Port

Additional options are also available when using SSL

T…	: 	URL of the trustpool

Z…	: 	NTP server address

#### NSO

Cisco provides an NSO PNP package. The configuration of the server to listen on all interfaces on port 9191 and without the of SSL will look like:

```
Connect to the NSO server (198.18.134.50 cisco/cisco)
[cisco@nso ~]$ ncs_cli -u admin -C
admin connected from 10.16.170.242 using ssh on nso
admin@ncs# config t
Entering configuration mode terminal
admin@ncs(config)# pnp server ip-address 0.0.0.0 port 9191 use-ssl false
admin@ncs(config)# commit
```

When SSL is enabled the PNP implementation on NSO allows to configure a secondary server which will only be used to provide the secure trustpool over http and after the trustpool is received by the client it will switch over to the secured server. 

The NSO PNP server has an internal map which can be configured to provide customized attributes and day0 configuration file for each device serial. When the device is contacting the NSO PNP server, it is checked if the device is present in the mapping list. If the device is not mapped, the server will put the serial in the unclaimed list.

```
admin@ncs# show pnp unclaimed
ID
-------------
9JLRL1RK4H6
```

We can add a mapping for that serial and it will get configured with the specific day0 file next time it will send a work request:

```
admin@ncs(config-map)# pnp map 9JLRL1RK4H6 serial 9JLRL1RK4H6 device-name 9JLRL1RK4H6 device-type cli ned-id cisco-ios mgmt-ip-address 198.18.1.31 port 22 username cisco password cisco day0-file day0.cfg
```

At the end of the process the device will be onboarded and sync-ed on NSO.



#### Configure the PNP agent on the device

When the device will boot it will contact the DHCP server and it will receive address of the PNP server. The PNP agent on the device can also be manually configured.

Example of configuring the agent on the CSR1Kv:

```
Router(config)#pnp profile cleur
Router(config-pnp-init)#transport http ipv4 198.18.134.50 port 9191
```

### Cisco POAP

On NXOS, very similar with ZTP, the POAP (PowerOn Auto Provisioning) relies on different DHCP options(66 and 67) for obtaining the URL of the configuration script. The process starts at the boot time and checks if a start-up config exits. If there is no start-up configuration on the switch, POAP starts a DHCP client on the management interface (on version 7.0(3)I5(1)) or all interfaces (starting with Cisco NX-OS Release 7.0(3)I5(2) and Cisco NX-OS Release 7.0(3)I6(1)) for an IP address and wait for a response. Together with the IP address the DHCP server will provide an HTTP/HTTPS/TFTP URL to download a PYTHON script using option 66 and 67. The POAP process will download the file, check the checksum of the file and if it is correct starts the execution of the script. 

<img src="img/poap_1.png" width="1000"/>
 
The script file must contain the checksum of the file itself on the top of the file. That means that in contrast with ZTP, whenever the script file is modified the checksum needs to be re-calculated and added to the file.
Configuration of the DHCP server (ISC-DHCP) to be able to provide both the server and the filename for download for the POAP protocol needs to contain 2 options:

```
#option 66
option tftp-server-name "http://1.2.3.4";

#option 67
option bootfile-name "test.py";
````

Note the fact that even if the name of the option 66 refers to a tftp-server, HTTP and HTTPS are also supported for downloading the script file.

Cisco provides examples of python scripts which may be adapted and used in the POAP process (for example for Nexus 9K: https://github.com/datacenter/nexus9000/blob/master/nx-os/poap/poap.py).

In the script file it is also possible to specify a required image version to be installed on the switch, ad the rest of the configuration is only applied after the image download&install and rebooting the switch.

